In [1]:
from utils import add_project_src_to_path

# Path adjustment helper
add_project_src_to_path()

In [ ]:
from core import settings

if settings.GROQ_API_KEY is None:
    print("GROQ_API_KEY not set, skipping LlamaGuard")
else:
    print("GROQ_API_KEY set successfully")

In [ ]:
from typing import Literal
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph, END, MessagesState
from schema.models import GroqModelName
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning, message=".*ConversationBufferMemory.*")

from core import get_model, settings

# --- State ---
class PreScreenState(MessagesState, total=False):
    result: str


# --- Load model (e.g., llama3-70b-8192 from Groq or OpenAI) ---
model = get_model(GroqModelName.LLAMA_33_70B)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# --- System Prompt / Role ---
system_prompt = SystemMessage(
    content="""
You are a job application pre-screener bot. Your job is to review candidate input and determine whether they are eligible for the next round.
Requirements:
- At least 3 years of experience
- Must know Python

You must respond only with a decision and a short explanation.
"""
)

# --- Initial candidate data (simulate input) ---
user_input = HumanMessage(
    content="Name: Alex, Email: alex@example.com, Experience: 4 years, Skills: Python, Django"
)


# --- Step: Evaluate eligibility using model ---
async def evaluate_candidate(state: PreScreenState, config: RunnableConfig) -> PreScreenState:
    messages = state["messages"]
    if not any(isinstance(m, SystemMessage) for m in messages):
        messages = [system_prompt] + messages
    result = await model.ainvoke(messages, config)
    return {"messages": [result], "result": result.content}


# --- Step: Log result and end ---
async def finalize(state: PreScreenState, config: RunnableConfig) -> PreScreenState:
    print("✅ Eligibility Check Complete:")
    print(state["result"])
    return state


# --- Graph Setup ---
graph = StateGraph(PreScreenState)
graph.add_node("evaluate_candidate", evaluate_candidate)
graph.add_node("finalize", finalize)

graph.set_entry_point("evaluate_candidate")
graph.add_edge("evaluate_candidate", "finalize")
graph.add_edge("finalize", END)

# --- Compile graph ---
app_graph = graph.compile()

# --- Run the workflow ---
print("🚀 Running Job Application Pre-Screening...\n")
result = await app_graph.ainvoke({"messages": [user_input]})

# --- Output result ---
print("\n🧠 Final Messages:")
for msg in result["messages"]:
    print(f"{msg.type.upper()}: {msg.content}")

🚀 Running Job Application Pre-Screening...

✅ Eligibility Check Complete:
**ELIGIBLE**

You meet the minimum experience requirement of 3 years, and have listed Python as one of your skills.

🧠 Final Messages:
HUMAN: Name: Alex, Email: alex@example.com, Experience: 4 years, Skills: Python, Django
AI: **ELIGIBLE**

You meet the minimum experience requirement of 3 years, and have listed Python as one of your skills.
